In [1]:
import os
import pandas as pd
import watchdog.events
import watchdog.observers
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

# Configure os parâmetros de monitoramento de arquivos
pasta_raiz = 'X:\\ARQUIVOS\\3810'

# Lista de nomes base de arquivo sem a data
nomes_base_arquivos = ['THourlyUsage-v2', 'GTerminalData-v2', 'GDailyStats-v2', 'SHourlyUsageByCarrierVNO'
                       , 'SPerformanceDownload', 'SPerformanceLatency', 'SPerformanceUpload', 'beam_fullness_by_hour'
                       , 'Speedtestresultsbyvno', 'T_CUST_Active_Accounts', 'T_CUST_Disconnects', 'T_CUST_New_Connects'
                       , 'T_CUST_New_Orders', 'T_CUST_Suspended', 'Cópia de P beams 2023']

formato_data = "%Y%m%d"  # Formato da data usado nos nomes dos arquivos

# Crie uma lista para rastrear arquivos com erros
arquivos_com_erros = []

# Crie um manipulador de eventos personalizado para lidar com eventos de arquivo
class ManipuladorEventosArquivo(watchdog.events.FileSystemEventHandler):
    def on_modified(self, event):
        for nome_base_arquivo in nomes_base_arquivos:
            for tipo_arquivo in encontrar_arquivos_correspondentes(pasta_raiz, nome_base_arquivo):
                processar_arquivo(tipo_arquivo, nome_base_arquivo)
                
def obter_requisito_tamanho(nome_base_arquivo):
    # Defina requisitos de tamanho e formato de data com base no nome_base_arquivo
    requisitos = {
        'THourlyUsage': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
          'Cópia de P beams 2023': {
            'tamanho': 1000000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'GTerminalData-v2': {
            'tamanho': 100000000,  # Tamanho mínimo para 'datafile' (75KB)
            'formato_data': "%Y%m%d",
        },
        'GDailyStats-v2': {
            'tamanho': 100000000,  # Tamanho mínimo para 'relatorio' (60KB)
            'formato_data': "%Y%m%d",
        },
        'SHourlyUsageByCarrierVNO': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SPerformanceDownload': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SPerformanceLatency': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SPerformanceUpload': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'beam_fullness_by_hour': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'Speedtestresultsbyvno': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'T_CUST_Active_Accounts': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'T_CUST_Disconnects': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'T_CUST_New_Connects': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'T_CUST_New_Orders': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'T_CUST_Suspended': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        
        # Adicione outros tipos de arquivo aqui, se necessário
    }
    
    requisito = requisitos.get(nome_base_arquivo)
    if requisito:
        return requisito['tamanho'], requisito['formato_data']
    else:
        return None, None
    
def encontrar_arquivos_correspondentes(pasta_raiz, nome_base_arquivo):
    arquivos_correspondentes = []
    hoje = datetime.now().strftime(formato_data)
    for nome_arquivo in os.listdir(pasta_raiz):
        if nome_arquivo.startswith(nome_base_arquivo) and nome_arquivo[len(nome_base_arquivo):].startswith(hoje):
            arquivos_correspondentes.append(os.path.join(pasta_raiz, nome_arquivo))
    return arquivos_correspondentes

def processar_arquivo(caminho_arquivo, nome_base_arquivo):
    print(f'Processando arquivo: {caminho_arquivo}')
    # Extrair a data do nome do arquivo
    nome_arquivo = os.path.basename(caminho_arquivo)
    data_arquivo = datetime.strptime(nome_arquivo[len(nome_base_arquivo):], formato_data)

    # Verificar o tamanho do arquivo
    requisito_tamanho = obter_requisito_tamanho(nome_base_arquivo)
    if requisito_tamanho is not None and os.path.getsize(caminho_arquivo) < requisito_tamanho:
        arquivos_com_erros.append(f'{nome_base_arquivo}{data_arquivo.strftime(formato_data)} - Tamanho muito pequeno')

    # Verificar colunas vazias usando o Pandas
    try:
        df = pd.read_csv(caminho_arquivo)
        colunas_vazias = df.columns[df.columns[df.isnull().any()]]
        if colunas_vazias.size > 0:
            arquivos_com_erros.append(f'{nome_base_arquivo}{data_arquivo.strftime(formato_data)} - Colunas vazias: {", ".join(colunas_vazias)}')
    except Exception as e:
        arquivos_com_erros.append(f'{nome_base_arquivo}{data_arquivo.strftime(formato_data)} - Erro: {str(e)}')
        


if __name__ == '__main__':
    # Iniciar o observador de sistema de arquivos
    observador = watchdog.observers.Observer()
    manipulador_eventos = ManipuladorEventosArquivo()
    observador.schedule(manipulador_eventos, path=pasta_raiz, recursive=False)
    observador.start()

    try:
        while True:
            pass
    except KeyboardInterrupt:
        observador.stop()
    observador.join()

    # Enviar um e-mail se houver arquivos com erros
    if arquivos_com_erros:
        destinatarios = ['gusdaniel.devmail@gmail.com', ]
        assunto = 'Arquivos com Erros'
        mensagem = '\n'.join(arquivos_com_erros)

        msg = MIMEMultipart()
        msg['De'] = ''
        msg['Para'] = ', '.join(destinatarios)
        msg['Assunto'] = assunto
        msg.attach(MIMEText(mensagem, 'plain'))

        servidor = smtplib.SMTP('smtp.example.com', 587)
        servidor.starttls()
        servidor.login('', '')
        servidor.sendmail('', destinatarios, msg.as_string())
        servidor.quit()
    print("Arquivos com erros:")
    for erro in arquivos_com_erros:
        print(erro)
    # Criar um arquivo de texto listando os arquivos com erros
    with open('log_erros.txt', 'w') as f:
        f.write('\n'.join(arquivos_com_erros))


Arquivos com erros:


In [3]:
def obter_requisito_tamanho(nome_base_arquivo):
    # Defina requisitos de tamanho e formato de data com base no nome_base_arquivo
    requisitos = {
        'TelebrasHourlyUsage': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
          'Cópia de Poligonos beams 2023': {
            'tamanho': 1000000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'GESACTerminalData-v2': {
            'tamanho': 100000000,  # Tamanho mínimo para 'datafile' (75KB)
            'formato_data': "%Y%m%d",
        },
        'GESACDailyStats-v2': {
            'tamanho': 100000000,  # Tamanho mínimo para 'relatorio' (60KB)
            'formato_data': "%Y%m%d",
        },
        'SGDCHourlyUsageByCarrierVNO': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SGDCPerformanceDownload': {
            'tamanho': 100000000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SGDCPerformanceLatency': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'SGDCPerformanceUpload': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'beam_fullness_by_hour': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'Speedtestresultsbyvno': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'TELEBRAS_CUST_Active_Accounts': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'TELEBRAS_CUST_Disconnects': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'TELEBRAS_CUST_New_Connects': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'TELEBRAS_CUST_New_Orders': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        'TELEBRAS_CUST_Suspended': {
            'tamanho': 50000,  # Tamanho mínimo para 'arquivo' (50KB)
            'formato_data': "%Y%m%d",
        },
        
  servidor.login('gtsslogerror@gmail.com', 'gtss#311090
                 # Lista de nomes base de arquivo sem a data
nomes_base_arquivos = ['TelebrasHourlyUsage-v2', 'GESACTerminalData-v2', 'GESACDailyStats-v2', 'SGDCHourlyUsageByCarrierVNO'
                       , 'SGDCPerformanceDownload', 'SGDCPerformanceLatency', 'SGDCPerformanceUpload', 'beam_fullness_by_hour'
                       , 'Speedtestresultsbyvno', 'TELEBRAS_CUST_Active_Accounts', 'TELEBRAS_CUST_Disconnects', 'TELEBRAS_CUST_New_Connects'
                       , 'TELEBRAS_CUST_New_Orders', 'TELEBRAS_CUST_Suspended', 'Cópia de Poligonos beams 2023']

SyntaxError: unterminated string literal (detected at line 65) (1082645744.py, line 65)